In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Create a list of sets of N random numbers
N = 3
inputs = torch.randn((10000,N),dtype=torch.float32).cuda()
outputs = inputs ** 2

In [ ]:
outputs.device

In [ ]:
inputs[0]

In [ ]:
outputs[0]

In [ ]:
# set up graph for loss curve, accuracy curve
fig_loss, ax_loss = plt.subplots()
fig_accu, ax_accu = plt.subplots()

# create a test set
test_inputs = torch.randn((1000,N),dtype=torch.float32).cuda()
test_outputs = test_inputs ** 2

In [ ]:
def weight_bias(plot_num):
    #updates plot number in case of multiple function calls in the same cell
    count = plot_num
    #attains weights of first layer then divides into components
    Weights1 = model.state_dict()['0.weight'].tolist()
    Weights1_A = [i[0] for i in Weights1]
    Weights1_B = [i[1] for i in Weights1]
    Weights1_C = [i[2] for i in Weights1]
    #attains bias of the first layer
    Bias1 = model.state_dict()['0.bias'].tolist()
    #attains weights of second layer then divides into components
    Weights2 = model.state_dict()['2.weight'].tolist()
    Weights2_A = [i[0] for i in Weights2]
    Weights2_B = [i[1] for i in Weights2]
    Weights2_C = [i[2] for i in Weights2]
    #attains bias of the second layer
    Bias2 = model.state_dict()['2.bias'].tolist()
    plt.figure(count)
    count += 1
    #I included the code for scatterplots as comments in case those are desired down the road
    #plt.scatter(Weights1_A, list(range(len(Weights1_A))), color = 'b')
    plt.hist(Weights1_A, [-1, -.75, -.5, -.25, 0, .25, .5, .75, 1], color = 'b')
    plt.xlabel("Weights1_A")
    plt.ylabel("# of instances")
    plt.figure(count)
    count += 1
    #plt.scatter(Weights1_B, list(range(len(Weights1_B))), color = 'g')
    plt.hist(Weights1_B, [-1, -.75, -.5, -.25, 0, .25, .5, .75, 1], color = 'g')
    plt.xlabel("Weights1_B")
    plt.ylabel("# of instances")
    plt.figure(count)
    count += 1
    #plt.scatter(Weights1_C, list(range(len(Weights1_C))), color = 'r')
    plt.hist(Weights1_C, [-1, -.75, -.5, -.25, 0, .25, .5, .75, 1], color = 'r')
    plt.xlabel("Weights1_C")
    plt.ylabel("# of instances")
    plt.figure(count)
    count += 1
    #plt.scatter(Bias1, list(range(len(Bias1))), color = 'c')
    plt.hist(Bias1, [-1.5, -1, -.5, 0, .5, 1, 1.5], color = 'c')
    plt.xlabel("Bias1")
    plt.ylabel("# of instances")
    plt.figure(count)
    count += 1
    #There are only 3 points for weights and bias in the 2nd layer so I left them as scatterplots
    plt.scatter(Weights2_A, list(range(len(Weights2_A))), color = 'm')
    plt.xlabel("Weights2_A")
    plt.ylabel("# of index")
    plt.figure(count)
    count += 1
    plt.scatter(Weights2_B, list(range(len(Weights2_B))), color = 'y')
    plt.xlabel("Weights2_B")
    plt.ylabel("# of index")
    plt.figure(count)
    count += 1
    plt.scatter(Weights2_C, list(range(len(Weights2_C))), color = 'k')
    plt.xlabel("Weights2_C")
    plt.ylabel("# of index")
    plt.figure(count)
    count += 1
    plt.scatter(Bias2, list(range(len(Bias2))), color = 'hotpink')
    plt.xlabel("Bias2")
    plt.ylabel("# of index")
    #returns a dictionary with all the saved weight and bias data
    return {"Weights1":Weights1, "Weights1_A":Weights1_A,"Weights1_B":Weights1_B,"Weights1_C":Weights1_C,"Bias1":Bias1,"Weights2":Weights2, "Weights2_A":Weights2_A,"Weights2_B":Weights2_B,"Weights2_C":Weights2_C,"Bias2":Bias2}


In [ ]:
# Check what proportion of the predictions falls within 0.01 absolute accuracy or 1% relative accuracy
def test_data(prediction, actual):
    "Output 1 if prediction is accurate enough and 0 otherwise"
    abs_err = abs(prediction-actual)
    rel_err = abs_err / actual
    if (abs_err < 0.01 or rel_err < 0.01):
        return 1
    else: return 0

v_test_data = np.vectorize(test_data) # This makes a vector function and avoids a for loop to improve performance

In [ ]:
import pprint

# Create a simple neural network
hidden_nodes = 100
model = torch.nn.Sequential(
    torch.nn.Linear(inputs.shape[1],hidden_nodes),
    torch.nn.ReLU(),
    # torch.nn.Linear(hidden_nodes,hidden_nodes),
    # torch.nn.ReLU(),
    torch.nn.Linear(hidden_nodes,outputs.shape[1])
)
model.cuda()

#print('Weights and bias values before training.')
#for label in model.state_dict().keys():
    #x = model.state_dict()[label]
    # print('\n{}: '.format(label))
    #pprint.pprint(x)

#print('\nCheck the neural network output before training:')
#with torch.no_grad():
    #model.eval()
    #test = torch.tensor([0.1, 0.2, 0.3]).cuda()
    #prediction = model(test)
    # print('input: {}, output: {}'.format(test,prediction))

initial_wb_dict = weight_bias(1)

In [ ]:
import math

# Get ready to train
model.train()

# Break the list up into smaller batches for more efficient training
numMiniBatch = int(math.floor(inputs.shape[0]/100.))
inputMiniBatches = inputs.chunk(numMiniBatch)
outputMiniBatches = outputs.chunk(numMiniBatch)

# Train the neural network
lossFunc = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

n_epochs = 1000
for epoch in range(n_epochs):
    # Print something every 10 epochs of training
    # Plot accuracy every 10 epochs
    if epoch%10 == 0:
        print('=>Starting {}/{} epochs.'.format(epoch+1,n_epochs))
        
        with torch.no_grad():
            model.eval()
            prediction_temp = model(test_inputs)
            score_temp = v_test_data(prediction_temp.cpu().detach().numpy(), test_outputs.cpu().detach().numpy())
            accu_temp = np.sum(score_temp) / torch.numel(test_inputs)
            ax_accu.plot(epoch, accu_temp, 'bo', markersize=1)
            ax_accu.set_xlabel('Epochs')
            ax_accu.set_ylabel('Accuracy')
        model.train()
            
    for minibatch in range(numMiniBatch):
        prediction = model(inputMiniBatches[minibatch])
        loss = lossFunc(prediction,outputMiniBatches[minibatch])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Plot two points on the loss graph for each epoch, one for testing and one for training.
    total_prediction = model(inputs)
    total_loss = lossFunc(total_prediction, outputs).item()
    test_total_prediction = model(test_inputs)
    test_total_loss = lossFunc(test_total_prediction, test_outputs).item()
    ax_loss.plot(epoch, total_loss, 'bo', markersize=1, label='Training')
    ax_loss.plot(epoch, test_total_loss, 'go', markersize=1, label='Testing')
    ax_loss.set_xlabel('Epochs')
    ax_loss.set_ylabel('Total Loss')
    ax_loss.yscale('log')
# ax_loss.legend() # I didn't find an easy way to add a legend for this particular graph
    
print ('Training done!')
final_wb_dict = weight_bias(1)

In [ ]:
fig_loss

In [ ]:
fig_accu

In [ ]:
# fig_loss.savefig('loss.pdf')

In [ ]:
# test a single input, not very useful
with torch.no_grad():
    model.eval()
    test = torch.tensor([0.1, 0.2, 0.3]).cuda()
    prediction = model(test)
    print('input: {}, prediction: {}'.format(test,prediction))

In [ ]:
# Check out the output by running the model in evaluation
# make some residual plots

fig, ax = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()


# this can be easily modified to plot output vs. input directly
with torch.no_grad():
    model.eval()
    for input in test_inputs:
        prediction = model(input)
        actual = input ** 2
        residual = actual - prediction
        ax.plot(input[0].item(),residual[0].item(), 'bo', markersize=1)
        ax.set_xlabel('1st coordinate input')
        ax.set_ylabel('1st coordinate error')
        ax2.plot(input[1].item(),residual[1].item(), 'go', markersize=1)
        ax2.set_xlabel('2nd coordinate input')
        ax2.set_ylabel('2nd coordinate error')
        ax3.plot(input[2].item(),residual[2].item(), 'co', markersize=1)
        ax3.set_xlabel('3rd coordinate input')
        ax3.set_ylabel('3rd coordinate error')

In [ ]:
# fig.savefig('1st-1st.pdf')
# fig2.savefig('2nd-2nd.pdf')
# fig3.savefig('3rd-3rd.pdf')

In [ ]:
# I didn't find a way to get around having to .cpu() first before converting to a numpy array
score = v_test_data(test_total_prediction.cpu().detach().numpy(), test_outputs.cpu().detach().numpy())

In [ ]:
# I realized that this actually treats the coordinates of the vectors as separate data points, so it's
# actually computing the proportion of accurate coordinates. To fix this I'll need to find the norm of the vectors
# and check accuracy on those. 
np.sum(score) / torch.numel(test_inputs)

In [ ]:
# Work out the details
x0 = test[0].item()
x1 = test[1].item()
x2 = test[2].item()

print('Input to neural network:')
print('x0={:7.4f}, x1={:7.4f},x2={:7.4f}'.format(x0,x1,x2))

# Check the math by hand
A = model.state_dict()['0.weight'].tolist()
a = model.state_dict()['0.bias'].tolist()

sum0 = A[0][0]*x0+A[0][1]*x1+A[0][2]*x2+a[0]
sum1 = A[1][0]*x0+A[1][1]*x1+A[1][2]*x2+a[1]
sum2 = A[2][0]*x0+A[2][1]*x1+A[2][2]*x2+a[2]
print('\nSums for first layer:')
print('{:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(sum0,A[0][0],x0,A[0][1],x1,A[0][2],x2,a[0]))
print('{:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(sum1,A[1][0],x0,A[1][1],x1,A[1][2],x2,a[1]))
print('{:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(sum2,A[2][0],x0,A[2][1],x1,A[2][2],x2,a[2]))

y0 = max(sum0,0)
y1 = max(sum1,0)
y2 = max(sum2,0)
print('\nApplying the ReLU to the sums:')
print('y0 = {:7.4f} = ReLU({:7.4f})'.format(y0,sum0))
print('y1 = {:7.4f} = ReLU({:7.4f})'.format(y1,sum1))
print('y2 = {:7.4f} = ReLU({:7.4f})'.format(y2,sum2))

B = model.state_dict()['2.weight'].tolist()
b = model.state_dict()['2.bias'].tolist()

z0 = B[0][0]*y0+B[0][1]*y1+B[0][2]*y2+b[0]
z1 = B[1][0]*y0+B[1][1]*y1+B[1][2]*y2+b[1]
z2 = B[2][0]*y0+B[2][1]*y1+B[2][2]*y2+b[2]
print('\nCalculating the final output:')
print('z0 = {:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(z0,B[0][0],y0,B[0][1],y1,B[0][2],y2,b[0]))
print('z1 = {:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(z1,B[1][0],y0,B[1][1],y1,B[1][2],y2,b[1]))
print('z2 = {:7.4f} = {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f} * {:7.4f} + {:7.4f}'.format(z2,B[2][0],y0,B[2][1],y1,B[2][2],y2,b[2]))